In [68]:
import sqlite3
import requests

In [87]:
poke_ai_url = 'https://pokeapi.co/api/v2/'
endpoints = {
    'pokemon': "pokemon/{0}/"
}

sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

In [70]:
print(endpoints['pokemon'].format(*['1']))

pokemon/1/


In [71]:
def endpoint_request(endpoint: str, params: list):
  response = requests.get(f"{poke_ai_url}{endpoints[endpoint]}".format(*params))
  print(response.status_code)
  return response

In [197]:
pokemons = [
    (
        pokemon_response.json().get('id')+50000,
        pokemon_response.json().get('name'),
        [stat['base_stat'] for stat in pokemon_response.json().get('stats') if (stat_info := stat['stat'])['name'] in ['hp', 'attack']]
    )
    for i in range(1,10)
    if (pokemon_response := endpoint_request('pokemon',*[str(i)])).status_code == 200
]

200
200
200
200
200
200
200
200
200


In [198]:
print(pokemons)

[(50001, 'bulbasaur', [45, 49]), (50002, 'ivysaur', [60, 62]), (50003, 'venusaur', [80, 82]), (50004, 'charmander', [39, 52]), (50005, 'charmeleon', [58, 64]), (50006, 'charizard', [78, 84]), (50007, 'squirtle', [44, 48]), (50008, 'wartortle', [59, 63]), (50009, 'blastoise', [79, 83])]


In [201]:
pokemon_stats = [(id,name,*stats) for id,name,stats in pokemons]

In [244]:
inserted_pokemons = ', '.join([str(pokemon) for pokemon in pokemon_stats])

In [203]:
print(inserted_pokemons)

(50001, 'bulbasaur', 45, 49), (50002, 'ivysaur', 60, 62), (50003, 'venusaur', 80, 82), (50004, 'charmander', 39, 52), (50005, 'charmeleon', 58, 64), (50006, 'charizard', 78, 84), (50007, 'squirtle', 44, 48), (50008, 'wartortle', 59, 63), (50009, 'blastoise', 79, 83)


In [204]:
conn = sqlite3.connect('pokemons.db')

In [205]:
cur = conn.cursor()

In [206]:
cur.execute('''
  create table pokemons (
    id int not null,
    name string not null,
    hp int not null,
    attack int not null,
    primary key (id)
  )
  '''
)

In [236]:
sql_insert = f"""
INSERT INTO pokemons (id,name,hp,attack)
VALUES
   {inserted_pokemons};
"""

In [208]:
print(sql_insert)


INSERT INTO pokemons (id,name,hp,attack)
VALUES
   (50001, 'bulbasaur', 45, 49), (50002, 'ivysaur', 60, 62), (50003, 'venusaur', 80, 82), (50004, 'charmander', 39, 52), (50005, 'charmeleon', 58, 64), (50006, 'charizard', 78, 84), (50007, 'squirtle', 44, 48), (50008, 'wartortle', 59, 63), (50009, 'blastoise', 79, 83);



In [209]:
cur.execute(sql_insert)

In [210]:
conn.commit()

In [211]:
[row for row in cur.execute('select * from pokemons')]

[(50001, 'bulbasaur', 45, 49),
 (50002, 'ivysaur', 60, 62),
 (50003, 'venusaur', 80, 82),
 (50004, 'charmander', 39, 52),
 (50005, 'charmeleon', 58, 64),
 (50006, 'charizard', 78, 84),
 (50007, 'squirtle', 44, 48),
 (50008, 'wartortle', 59, 63),
 (50009, 'blastoise', 79, 83)]

## **Inserir pokemons**

Adiciona os pokemons na API

In [227]:
poke_stats_request = [
    {
        'id':0,
        'name':row[1],
        'account': {
            'id':0,
            'number': f"{str(row[0])}_{row[1]}",
            'agency': f"{str(row[0])}_{row[1]}",
            'balance': row[2],
            'limit': row[3],
        },
        "card": {
          "id": 0,
          'number': f"{str(row[0])}_{row[1]}",
          "limit": row[3]
        },
        'features': [],
        'news': [],
    }
    for row in cur.execute('select * from pokemons')
]

In [228]:
print(poke_stats_request)

[{'id': 0, 'name': 'bulbasaur', 'account': {'id': 0, 'number': '50001_bulbasaur', 'agency': '50001_bulbasaur', 'balance': 45, 'limit': 49}, 'card': {'id': 0, 'number': '50001_bulbasaur', 'limit': 49}, 'features': [], 'news': []}, {'id': 0, 'name': 'ivysaur', 'account': {'id': 0, 'number': '50002_ivysaur', 'agency': '50002_ivysaur', 'balance': 60, 'limit': 62}, 'card': {'id': 0, 'number': '50002_ivysaur', 'limit': 62}, 'features': [], 'news': []}, {'id': 0, 'name': 'venusaur', 'account': {'id': 0, 'number': '50003_venusaur', 'agency': '50003_venusaur', 'balance': 80, 'limit': 82}, 'card': {'id': 0, 'number': '50003_venusaur', 'limit': 82}, 'features': [], 'news': []}, {'id': 0, 'name': 'charmander', 'account': {'id': 0, 'number': '50004_charmander', 'agency': '50004_charmander', 'balance': 39, 'limit': 52}, 'card': {'id': 0, 'number': '50004_charmander', 'limit': 52}, 'features': [], 'news': []}, {'id': 0, 'name': 'charmeleon', 'account': {'id': 0, 'number': '50005_charmeleon', 'agency'

In [287]:
creation_responses = [
    requests.post(
      f"{sdw2023_api_url}/users",
      headers={'accept':'*/*','Content-Type':'application/json'},
      data=json.dumps(item)
    )
    for item in poke_stats_request
]

In [230]:
[
    json.dumps(item)
    for item in poke_stats_request
]

['{"id": 0, "name": "bulbasaur", "account": {"id": 0, "number": "50001_bulbasaur", "agency": "50001_bulbasaur", "balance": 45, "limit": 49}, "card": {"id": 0, "number": "50001_bulbasaur", "limit": 49}, "features": [], "news": []}',
 '{"id": 0, "name": "ivysaur", "account": {"id": 0, "number": "50002_ivysaur", "agency": "50002_ivysaur", "balance": 60, "limit": 62}, "card": {"id": 0, "number": "50002_ivysaur", "limit": 62}, "features": [], "news": []}',
 '{"id": 0, "name": "venusaur", "account": {"id": 0, "number": "50003_venusaur", "agency": "50003_venusaur", "balance": 80, "limit": 82}, "card": {"id": 0, "number": "50003_venusaur", "limit": 82}, "features": [], "news": []}',
 '{"id": 0, "name": "charmander", "account": {"id": 0, "number": "50004_charmander", "agency": "50004_charmander", "balance": 39, "limit": 52}, "card": {"id": 0, "number": "50004_charmander", "limit": 52}, "features": [], "news": []}',
 '{"id": 0, "name": "charmeleon", "account": {"id": 0, "number": "50005_charmele

In [231]:
[i.text for i in creation_responses]

['{"id":2915,"name":"bulbasaur","account":{"id":3088,"number":"50001_bulbasaur","agency":"50001_bulbasaur","balance":45,"limit":49},"card":{"id":2821,"number":"50001_bulbasaur","limit":49},"features":[],"news":[]}',
 '{"id":2916,"name":"ivysaur","account":{"id":3089,"number":"50002_ivysaur","agency":"50002_ivysaur","balance":60,"limit":62},"card":{"id":2822,"number":"50002_ivysaur","limit":62},"features":[],"news":[]}',
 '{"id":2917,"name":"venusaur","account":{"id":3090,"number":"50003_venusaur","agency":"50003_venusaur","balance":80,"limit":82},"card":{"id":2823,"number":"50003_venusaur","limit":82},"features":[],"news":[]}',
 '{"id":2918,"name":"charmander","account":{"id":3091,"number":"50004_charmander","agency":"50004_charmander","balance":39,"limit":52},"card":{"id":2824,"number":"50004_charmander","limit":52},"features":[],"news":[]}',
 '{"id":2919,"name":"charmeleon","account":{"id":3092,"number":"50005_charmeleon","agency":"50005_charmeleon","balance":58,"limit":64},"card":{"

In [243]:
account_insert = [(account['id'],account['name'],account['account']['id'],account['card']['id'],account['account']['balance']) for i in creation_responses if (account := json.loads(i.text))]

In [245]:
inserted_account = ', '.join([str(account) for account in account_insert])
print(inserted_account)

(2915, 'bulbasaur', 3088, 2821, 45), (2916, 'ivysaur', 3089, 2822, 60), (2917, 'venusaur', 3090, 2823, 80), (2918, 'charmander', 3091, 2824, 39), (2919, 'charmeleon', 3092, 2825, 58), (2920, 'charizard', 3093, 2826, 78), (2921, 'squirtle', 3094, 2827, 44), (2922, 'wartortle', 3095, 2828, 59), (2923, 'blastoise', 3096, 2829, 79)


In [241]:
cur.execute('''
  create table pokemon_accounts (
    id int not null,
    pokemon string not null,
    account_id int not null,
    card_id int not null,
    hp_balance int not null,
    primary key (id)
  )
  '''
)

In [246]:
sql_insert = f"""
INSERT INTO pokemon_accounts (id,pokemon,account_id,card_id,hp_balance)
VALUES
   {inserted_account};
"""

In [247]:
cur.execute(sql_insert)

In [248]:
[row for row in cur.execute('select * from pokemon_accounts')]

[(2915, 'bulbasaur', 3088, 2821, 45),
 (2916, 'ivysaur', 3089, 2822, 60),
 (2917, 'venusaur', 3090, 2823, 80),
 (2918, 'charmander', 3091, 2824, 39),
 (2919, 'charmeleon', 3092, 2825, 58),
 (2920, 'charizard', 3093, 2826, 78),
 (2921, 'squirtle', 3094, 2827, 44),
 (2922, 'wartortle', 3095, 2828, 59),
 (2923, 'blastoise', 3096, 2829, 79)]

**Battle system**

In [346]:
def update_pokemon(accounts: list, first_pokemon: int, second_pokemon: int):
  if accounts[first_pokemon][5] > accounts[second_pokemon][5]:
    news = f"{accounts[first_pokemon][1]} ganhou contra {accounts[second_pokemon][1]}, {accounts[second_pokemon][1]} perde {match_points} pontos do seu saldo!"
    account_number = accounts[second_pokemon][0]
    pokemon = accounts[second_pokemon]
  else:
    news = f"{accounts[second_pokemon][1]} ganhou contra {accounts[first_pokemon][1]}, {accounts[first_pokemon][1]} perde {match_points} pontos do seu saldo!"
    account_number = accounts[first_pokemon][0]
    pokemon = accounts[first_pokemon]

  print(news)
  print(f"Account to be updated {account_number}")

  update_request = json.loads(requests.get(f"{sdw2023_api_url}/users/{account_number}").text)

  update_request['news'].append(
    {
      "id": 0,
      "icon": "a",
      "description": news
    }
  )
  if update_request['account']['balance'] - match_points > 0:
    update_request['account']['balance'] -= match_points
  else:
    update_request['account']['balance'] = 0
    update_request['news'].append(
        {
        "id": 0,
        "icon": "a",
        "description": f"{update_request['name']} desmaiou!"
      }

    )

  print(update_request)

  return requests.put(
    f"{sdw2023_api_url}/users/{account_number}",
    headers={'accept':'*/*','Content-Type':'application/json'},
    data=json.dumps(update_request)
  )

In [382]:
import random

match_points = 5

accounts = [
    row
    for row in cur.execute(
        '''
          select
            pa.*,
            p.attack
          from
            pokemon_accounts pa join pokemons p on pa.pokemon = p.name
        '''
    )
]
#print(accounts)

first_pokemon = random.randint(0, 8)
second_pokemon = random.randint(0, 8)

while first_pokemon == second_pokemon:
  second_pokemon = random.randint(0, 8)

print(first_pokemon, second_pokemon)

result = update_pokemon(accounts, first_pokemon, second_pokemon)

print(result.status_code, result.text)

6 3
charmander ganhou contra squirtle, squirtle perde 5 pontos do seu saldo!
Account to be updated 2921
{'id': 2921, 'name': 'squirtle', 'account': {'id': 3094, 'number': '50007_squirtle', 'agency': '50007_squirtle', 'balance': 0, 'limit': 48.0}, 'card': {'id': 2827, 'number': '50007_squirtle', 'limit': 48.0}, 'features': [], 'news': [{'id': 5854, 'icon': 'a', 'description': 'wartortle ganhou contra squirtle, squirtle perde 5 pontos do seu saldo!'}, {'id': 5857, 'icon': 'a', 'description': 'bulbasaur ganhou contra squirtle, squirtle perde 5 pontos do seu saldo!'}, {'id': 5859, 'icon': 'a', 'description': 'wartortle ganhou contra squirtle, squirtle perde 5 pontos do seu saldo!'}, {'id': 5860, 'icon': 'a', 'description': 'venusaur ganhou contra squirtle, squirtle perde 5 pontos do seu saldo!'}, {'id': 5862, 'icon': 'a', 'description': 'wartortle ganhou contra squirtle, squirtle perde 5 pontos do seu saldo!'}, {'id': 5868, 'icon': 'a', 'description': 'charizard ganhou contra squirtle, squ